# Generate a candlestick charts

In [0]:
# Install Kaleido for saving interactive plots as static images.
!pip install kaleido

In [0]:
%run "../utils/mount_configuration"

In [0]:
# Load evaluation date from 'tickers_quality_evaluation' task
evaluation_date = dbutils.jobs.taskValues.get(taskKey="tickers_quality_evaluation", key="evaluation_date", debugValue="2025-03-17")

In [0]:
from pyspark.sql.functions import col

warren_buffet_stocks_selection_df = spark.read \
    .format('delta') \
    .load(f'{presentation_folder_path}/warren_buffet_stocks_selection') \
    .filter(col('date') == evaluation_date) \
    .select('ticker') 
                                              
# display(warren_buffet_stocks_selection_df)

In [0]:
benjamin_graham_stocks_selection_df = spark.read \
    .format('delta') \
    .load(f'{presentation_folder_path}/benjamin_graham_stocks_selection') \
    .filter(col('date') == evaluation_date) \
    .select('ticker') 

# display(benjamin_graham_stocks_selection_df)

In [0]:
merged_stocks_selection_df = warren_buffet_stocks_selection_df \
    .union(benjamin_graham_stocks_selection_df)

# display(merged_stocks_selection_df)

In [0]:
from pyspark.sql.functions import col

# Load stock price data for selected tickers and filter based on the tickers list.
tickers_list = [row.ticker for row in merged_stocks_selection_df.select("ticker").distinct().collect()]

tickets_price_df = spark.read.format("delta") \
                            .load(f"{processed_folder_path}/tickers_price") \
                            .filter(col("ticker").isin(tickers_list))

# display(tickets_price_df)

In [0]:
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

# Creation directory in which will be stored all files for each evalution date 
directory = f'{presentation_folder_path}/investment_recommendations/{evaluation_date}'
dbutils.fs.mkdirs(directory)

# Convert PySpark DataFrame to Pandas
df_pandas = tickets_price_df.toPandas()

# Convert 'date' column to datetime format
df_pandas['date'] = pd.to_datetime(df_pandas['date'])

# Iterate over unique tickers
for ticker_symbol in tickers_list:
    # Filter data for the given ticker
    df_filtered = df_pandas[(df_pandas["ticker"] == ticker_symbol) & (df_pandas["date"] >= datetime.today().replace(year=datetime.today().year - 1))]
    
    # Sort by date
    df_filtered = df_filtered.sort_values("date")
    
    # Create candlestick chart
    fig = go.Figure(data=[go.Candlestick(
        x=df_filtered["date"],
        open=df_filtered["open_price"],
        high=df_filtered["highest_price"],
        low=df_filtered["lowest_price"],
        close=df_filtered["close_price"],
        increasing_line_color='red', 
        decreasing_line_color='green'
    )])

    # Configure chart
    fig.update_layout(
        title=f'Candlestick Chart for {ticker_symbol}',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        template="plotly_dark"
    )

    #Define filename and whole filepath for the chart
    plot_filename = f'{ticker_symbol}_candlestick_chart.png'
    plot_filepath = f'/dbfs/{directory}/{plot_filename}'

    fig.write_image(plot_filepath)

    # Display chart
    fig.show()